wandb: Downloading large artifact final_chatbot_checkpoint:v0, 288.27MB. 2 files... 
wandb:   2 of 2 files downloaded.  
Done. 0:0:1.3


In [15]:
# necessary packages for inference
import os
import math
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import wandb
from utils import Voc
from dataset import loadPrepareData, trimRareWords, split_dataset, get_dataloader
from model import EncoderRNN, LuongAttnDecoderRNN
from evaluate import GreedySearchDecoder
from datetime import datetime
from utils import *
from dataset import *
from model import *
from evaluate import *

In [10]:
import wandb
import os
import torch
wandb.init()



# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

artifact = wandb.use_artifact("abhi1199-city-university-of-london/seq2seqRNNXAtten_last/final_chatbot_checkpoint:v0", type="model")
artifact_dir = artifact.download()

checkpoint_path = os.path.join(artifact_dir, "final_checkpoint.tar")
voc_path = os.path.join(artifact_dir, "voc.pkl")

# Load checkpoint
checkpoint = torch.load(checkpoint_path, map_location=device)


wandb: Downloading large artifact final_chatbot_checkpoint:v0, 288.27MB. 2 files... 
wandb:   2 of 2 files downloaded.  
Done. 0:0:1.4


In [11]:
with open(voc_path, "rb") as f:
    voc = pickle.load(f)

In [17]:
# model configuration for inferecne
hidden_size = checkpoint['embedding_state']['weight'].shape[1]
embedding = nn.Embedding(voc.num_words, hidden_size)
embedding.load_state_dict(checkpoint['embedding_state'])
embedding = embedding.to(device)

encoder = EncoderRNN(hidden_size, embedding, n_layers=2, dropout=0.1).to(device)
decoder = LuongAttnDecoderRNN("dot", embedding, hidden_size, voc.num_words, n_layers=2, dropout=0.1).to(device)

encoder.load_state_dict(checkpoint['encoder_state'])
decoder.load_state_dict(checkpoint['decoder_state'])

encoder.eval()
decoder.eval()

# Initialize GreedySearchDecoder
searcher = GreedySearchDecoder(encoder, decoder)

# Chat loop
def chat():
    print("Hello! Financeot is ready to solve your query! Type 'quit' to exit.")
    while True:
        try:
            input_sentence = input("> ")
            if input_sentence.lower() in ["quit", "q"]:
                break
            input_sentence = normalizeString(input_sentence)
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            output_words = [w for w in output_words if w not in ["EOS", "PAD"]]
            print("Bot:", ' '.join(output_words))
        except KeyError:
            print("Oops! Encountered unknown word. Try again.")

if __name__ == "__main__":
    chat()


Hello! Financeot is ready to solve your query! Type 'quit' to exit.
Bot: some you can have a successful mobile of parts a test that can help you as much as possible . .
Bot: i believe you believe and what you have that can have you been . it is always good to have have you to have . .
Bot: there are three players you need to take a few steps . .
Bot: the criteria for you for is as follows as of and and and and . and .
Bot: it is not a common step to make it easy to pay . you are here to you pay about your money . you you you .


In [20]:
def chat():
    print("Hello! Financeot is ready to solve your query! Type 'quit' to exit.")
    while True:
        try:
            input_sentence = input("> ")
            if input_sentence.lower() in ["quit", "q"]:
                break
            print("You:", input_sentence)  # 👈 this line makes it visible
            input_sentence = normalizeString(input_sentence)
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            output_words = [w for w in output_words if w not in ["EOS", "PAD"]]
            print("Bot:", ' '.join(output_words))
        except KeyError:
            print("Oops! Encountered unknown word. Try again.")


if __name__ == "__main__":
    chat()


Hello! Financeot is ready to solve your query! Type 'quit' to exit.
You: Why do some stocks have a higher margin requirement?
Bot: some you can have a successful mobile of parts a test that can help you as much as possible . .
You: why do I need an emergency fund if I already have investments?
Bot: i believe you believe and what you have that can have you been . it is always good to have have you to have . .
You: How would bonds fare if interest rates rose?
Bot: there are three players you need to take a few steps . .
You: Is it possible to make money by getting a mortgage?
Bot: it is not a common step to make it easy to pay . you are here to you pay about your money . you you you .
You: What can cause rent prices to fall?
Bot: . have a comprehensive type of reasons to the task or open a few situation . . . .
